In [1]:
#Mount the drive
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Replace 'your_folder_name' with the actual name of your folder
folder_path = '/content/drive/MyDrive/Research Project/DiffusionModels_DDPM_DDIM'

# Check if the folder exists
if os.path.exists(folder_path):
  print(f"Folder '{folder_path}' mounted successfully!")
else:
  print(f"Folder '{folder_path}' not found. Please check the path.")

# Now you can access files within the mounted folder
# For example, to list files in the folder:
os.listdir(folder_path)

Folder '/content/drive/MyDrive/Research Project/DiffusionModels_DDPM_DDIM' mounted successfully!


['.git',
 '__pycache__',
 'train',
 'Sampling_sprites.ipynb',
 'train.py',
 'Sampling_backbone.ipynb',
 'train_attn.py',
 'train_hddn.py',
 'spatial_helper.py',
 'unet_attention.py',
 'unet_test.ipynb',
 'diffusion_utils (2).py',
 'diffusion_utils (1).py',
 'diffusion_utils.py',
 'Sampling.ipynb',
 'L4_FastSampling.ipynb',
 'output_image.jpg',
 'diffusion_01_score.ipynb',
 'helper_plot.py',
 'train_context.py',
 'train_metadata.csv',
 'train_labels.csv',
 'gitignore',
 'README.md',
 'GenerateLabels.ipynb',
 'Sampling_attn.ipynb',
 'Sampling_attn_M.ipynb',
 'GenerateLabels_M.ipynb',
 'GenerateLabels (1).ipynb',
 'wind_3D16X16.npy',
 'wind_label_3D16X16.npy',
 'wind_label_3D128X128.npy',
 'wind_3D128X128.npy',
 'resnet_helper.py',
 'Sampling (1).ipynb',
 'Copy of Sampling (1).ipynb',
 'Copy of GenerateLabels (1).ipynb']

In [3]:
import numpy as np
import pandas as pd
from PIL import Image
train_file_path = os.path.join(folder_path, "train_labels.csv")
train_file_path_meta = os.path.join(folder_path, "train_metadata.csv")
labels = pd.read_csv(train_file_path)
metadata = pd.read_csv(train_file_path_meta)
array_2000 =[]

In [4]:
labels.head()

,Image ID,Wind Speed
0,nhe_000,34
1,nhe_001,34
2,nhe_002,34
3,nhe_003,33
4,nhe_004,32


In [5]:
metadata.head()

,Image ID,Storm ID,Relative Time,Ocean
0,nhe_000,nhe,0,2
1,nhe_001,nhe,1800,2
2,nhe_002,nhe,3600,2
3,nhe_003,nhe,5402,2
4,nhe_004,nhe,9001,2


In [6]:
#Check 1 image
filename = "ggv_278.jpg"
image_file_path=os.path.join(folder_path,"train/", filename)
img = Image.open(image_file_path)
image_array = np.array(img)
image_array.shape[0]

366

In [7]:
def channel3(img, size):
    empty_3d_array = np.empty((size, size, 3))
    empty_3d_array[:,:,0]=np.array(img)
    empty_3d_array[:,:,1]=np.array(img)
    empty_3d_array[:,:,2]=np.array(img)
    return empty_3d_array


In [8]:
def channel1(img):
    empty_1d_array = np.empty((16, 16, 1))
    empty_1d_array[:,:,0]=np.array(img)
    return empty_1d_array

In [9]:
def crop_center(image, new_width, new_height):
    # Get the current dimensions of the image
    width, height = image.size

    # Calculate the coordinates for the crop
    left = (width - new_width) // 2
    top = (height - new_height) // 2
    right = (width + new_width) // 2
    bottom = (height + new_height) // 2

    # Perform the crop
    cropped_image = image.crop((left, top, right, bottom))

    return cropped_image




In [12]:
# CHANGING IMAGE SIZE FROM 128 TO 16

In [13]:
from tqdm import tqdm

def process_images(folder_path):
    image_array = []
    for filename in tqdm(os.listdir(folder_path), desc="Processing images"):
        if filename.endswith(".jpg"):
            file_path = os.path.join(folder_path, filename)
            size = 16     # changing the image size from 128 to 16
            try:
                img = Image.open(file_path)
                cropped_image = crop_center(img, size, size)  # Crop to 16 x 16
                image_array.append(channel3(cropped_image, size))
                img.close()
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return image_array

### Find the common images between label.csv file and `/train` folder. Save the wind_speed as a dict to be used to create the image_array and labels_array in the later codes.

In [14]:
folder_path_image= os.path.join(folder_path,"train/")
filenames = os.listdir(folder_path_image)
image_names = []  # Create an empty list to store image names
df =labels.copy()

image_filenames = [f for f in os.listdir(folder_path_image) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
# Extract the 'Image ID' column from the DataFrame
image_ids_df = df['Image ID'].astype(str).tolist()  # Convert to strings for comparison

# Convert image filenames to IDs by removing extensions
image_ids_files = [os.path.splitext(f)[0] for f in image_filenames]
# Find common and unique elements
common_ids = list(set(image_ids_df) & set(image_ids_files))
unique_ids_df = list(set(image_ids_df) - set(image_ids_files))
unique_ids_files = list(set(image_ids_files) - set(image_ids_df))

# Print the results
print(f"Number of common Image IDs: {len(common_ids)}")
print(f"Number of unique Image IDs in DataFrame: {len(unique_ids_df)}")
print(f"Number of unique Image IDs in files: {len(unique_ids_files)}")

wind_speeds = {}

# Iterate through common IDs and retrieve wind speeds
for image_id in common_ids:
    wind_speed = df.loc[df['Image ID'] == image_id, 'Wind Speed'].iloc[0]
    wind_speeds[image_id] = wind_speed


Number of common Image IDs: 24677
Number of unique Image IDs in DataFrame: 45580
Number of unique Image IDs in files: 0


In [15]:
def labels_process(value_loc):
    storm_array=[]
    # name_to_find = image_id
    # result_loc = labels.loc[labels['Image ID'] == name_to_find, 'Wind Speed'].values
    # value_loc=result_loc[0]
    if ((value_loc>=15) & (value_loc<=45)):
        storm_array = switch_case(1)
    elif ((value_loc>45) & (value_loc<=80)):
        storm_array = switch_case(2)
    elif (value_loc>80 & value_loc<=110):
        storm_array = switch_case(3)
    elif (value_loc>110 & value_loc<=150):
        storm_array = switch_case(4)
    elif (value_loc>150 & value_loc<=190):
        storm_array = switch_case(5)
    return storm_array

In [16]:
def switch_case(argument):
    return {
        1: [1,0,0,0,0],
        2: [0,1,0,0,0],
        3: [0,0,1,0,0],
        4: [0,0,0,1,0],
        5: [0,0,0,0,1]
    }.get(argument, "Invalid option")

### use 10% of images to check the code

In [17]:
from tqdm import tqdm
import random

def process_images(folder_path,wind_speeds):
    image_array = []
    filenames = os.listdir(folder_path)
    num_files_to_process = int(len(filenames) * 0.1)  # Calculate 10%
    wind_speed_array=[]
    # Wrap selection process with tqdm
    with tqdm(total=num_files_to_process, desc="Selecting files") as pbar_selection:
        selected_filenames = random.sample(filenames, num_files_to_process)
        pbar_selection.update(num_files_to_process) # Update selection progress bar

    # Wrap processing loop with tqdm
    with tqdm(total=num_files_to_process, desc="Processing images") as pbar_processing:
        for filename in selected_filenames:
            if filename.endswith(".jpg"):
                file_path = os.path.join(folder_path, filename)
                size = 128
                try:
                    img = Image.open(file_path)
                    cropped_image = crop_center(img, size, size)
                    image_array.append(channel3(cropped_image, size))
                    wind_speed_array.append(labels_process(wind_speeds[image_id]))  # Add wind speed to list
                    img.close()
                except Exception as e:
                    print(f"Error processing {filename}: {e}")
                finally:
                    pbar_processing.update(1) # Update processing progress bar
    return image_array,wind_speed_array

In [18]:
def process_images_two_sets(folder_path, wind_speeds):
    image_array = []
    wind_speed_array = []

    filenames = os.listdir(folder_path)
    num_files_to_process = int(len(filenames) * 0.1)  # Process 10%

    # Random selection
    with tqdm(total=num_files_to_process, desc="Selecting files") as pbar_selection:
        selected_filenames = random.sample(filenames, num_files_to_process)
        pbar_selection.update(num_files_to_process)

    # Image processing
    with tqdm(total=num_files_to_process, desc="Processing images") as pbar_processing:
        for filename in selected_filenames:
            if filename.endswith(".jpg"):
                file_path = os.path.join(folder_path, filename)
                size = 16
                try:
                    image_id = os.path.splitext(filename)[0]  # Get ID from filename
                    img = Image.open(file_path)

                    # --- Original Image ---
                    cropped_image = crop_center(img, size, size)
                    original = channel3(cropped_image, size)
                    image_array.append(original)
                    wind_speed_array.append(labels_process(wind_speeds[image_id]))

                    # --- Rotated Image ---
                    angle = random.choice([90, 180, 270])
                    rotated_image = cropped_image.rotate(angle)
                    rotated = channel3(rotated_image, size)
                    image_array.append(rotated)
                    wind_speed_array.append(labels_process(wind_speeds[image_id]))

                    img.close()

                except Exception as e:
                    print(f"Error processing {filename}: {e}")
                finally:
                    pbar_processing.update(1)

    return image_array, wind_speed_array

In [19]:
image_array,labels_array= process_images_two_sets(folder_path_image,wind_speeds)

Processing images: 100%|██████████| 2467/2467 [32:59<00:00,  1.25it/s]


In [20]:
file_path_label = "wind_label_3D16X16.npy"
file_path = "wind_3D16X16.npy"
np.save(file_path, image_array)
np.save(file_path_label, labels_array)

In [21]:
import os
import pandas as pd

# Assuming 'metadata' is your DataFrame and 'folder_path_image' is the path to your images
image_folder = folder_path_image  # Replace with your image folder path
df = metadata  # Assuming 'metadata' is your DataFrame

# Get a list of image filenames from the folder
image_filenames = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

# Extract the 'Image ID' column from the DataFrame
image_ids_df = df['Image ID'].astype(str).tolist()  # Convert to strings for comparison

# Convert image filenames to IDs by removing extensions
image_ids_files = [os.path.splitext(f)[0] for f in image_filenames]

# Find common and unique elements
common_ids = list(set(image_ids_df) & set(image_ids_files))
unique_ids_df = list(set(image_ids_df) - set(image_ids_files))
unique_ids_files = list(set(image_ids_files) - set(image_ids_df))

# Print the results
print(f"Number of common Image IDs: {len(common_ids)}")
print(f"Number of unique Image IDs in DataFrame: {len(unique_ids_df)}")
print(f"Number of unique Image IDs in files: {len(unique_ids_files)}")

# Optional: Print the unique IDs
if unique_ids_df:
    print("\nUnique Image IDs in DataFrame:")
    print(unique_ids_df)
if unique_ids_files:
    print("\nUnique Image IDs in files:")
    print(unique_ids_files)

Number of common Image IDs: 24677
Number of unique Image IDs in DataFrame: 45580
Number of unique Image IDs in files: 0

Unique Image IDs in DataFrame:
['byg_060', 'zcu_000', 'hht_215', 'amd_007', 'xkr_049', 'gux_116', 'xmf_076', 'amd_126', 'amn_113', 'dnd_002', 'dkb_053', 'blq_234', 'cxh_004', 'bkh_173', 'vtw_058', 'dce_332', 'bkf_104', 'ojv_285', 'kza_102', 'eoi_368', 'tmv_052', 'zvv_218', 'rzq_099', 'whm_066', 'djr_041', 'psz_020', 'kyq_080', 'oup_095', 'psz_070', 'hrw_223', 'aek_001', 'zzp_083', 'gkd_001', 'zgi_124', 'wbb_174', 'ijc_152', 'ezh_209', 'tnn_104', 'tlc_095', 'kxm_027', 'wzu_089', 'cki_154', 'gux_020', 'sth_092', 'dky_083', 'vye_407', 'eml_010', 'fnu_039', 'iaz_013', 'btq_036', 'woc_258', 'iiz_044', 'pcq_012', 'ryg_063', 'xmf_260', 'zvb_031', 'kqu_177', 'dli_164', 'vye_102', 'zny_318', 'qdh_124', 'uhd_035', 'xgr_140', 'fpc_029', 'pvj_354', 'zny_238', 'ewt_053', 'ing_101', 'fcr_272', 'ztb_174', 'amn_186', 'kqu_335', 'eoi_071', 'zkg_067', 'pcq_015', 'orp_181', 'amn_300', 

In [22]:
print(len(labels_array))

4934
